In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bayesflow as bf

workflow = bf.BasicWorkflow(
    inference_network=bf.networks.PointInferenceNetwork(scores=dict(mvn=bf.scores.MultivariateNormalScore())),
    summary_network=bf.networks.TimeSeriesNetwork(),
    inference_variables=["parameters"],
    summary_variables=["observables"],
    simulator=bf.simulators.SIR()
)

history = workflow.fit_online(epochs=32, batch_size=32, num_batches_per_epoch=64)


2025-07-28 18:56:14.061351: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-28 18:56:14.064535: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-28 18:56:14.072793: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753721774.085987 1625127 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753721774.089846 1625127 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-28 18:56:14.105485: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Epoch 1/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 10s 56ms/step - loss: 2.6824 - mvn/inference_mvn: 2.6824
Epoch 2/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: 1.4767 - mvn/inference_mvn: 1.4767
Epoch 3/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 60ms/step - loss: 0.9426 - mvn/inference_mvn: 0.9426
Epoch 4/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: 0.7816 - mvn/inference_mvn: 0.7816
Epoch 5/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: 0.6035 - mvn/inference_mvn: 0.6035
Epoch 6/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: 0.3166 - mvn/inference_mvn: 0.3166
Epoch 7/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: -0.2796 - mvn/inference_mvn: -0.2796
Epoch 8/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: -0.6342 - mvn/inference_mvn: -0.6342
Epoch 9/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/step - loss: -0.7620 - mvn/inference_mvn: -0.7620
Epoch 10/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 59ms/step - loss: -0.9120 - mvn/inference_mvn: -0.9120
Epoch 11/32
64/64 ━━━━━━━━━━━━━━━━━━━━ 4s 58ms/ste

In [3]:
import numpy as np

samples = workflow.simulator.sample(4)

In [4]:
np.set_printoptions(suppress=True)

In [11]:
estimates = workflow.estimate(conditions=samples)
cov_chol_inv = estimates["parameters"]["mvn"]["cov_chol_inv"
]
np.linalg.inv(np.einsum("...ij,...ik->...jk", cov_chol_inv, cov_chol_inv))

array([[[0.00017129, 0.00001643],
        [0.00001643, 0.00001212]],

       [[0.00036046, 0.00006673],
        [0.00006673, 0.00004149]],

       [[0.00017934, 0.00001579],
        [0.00001579, 0.0000106 ]],

       [[0.00079195, 0.00002988],
        [0.00002988, 0.0000155 ]]])

In [12]:
e = np.linalg.inv(np.einsum("...ij,...ik->...jk", cov_chol_inv, cov_chol_inv))

In [13]:
l = []
for s in workflow.sample(conditions=samples, num_samples=100_000)["parameters"]:
    l.append(np.cov(s, rowvar=False))
l = np.array(l)
l, e, e-l

(array([[[0.00017213, 0.0000162 ],
         [0.0000162 , 0.00001206]],
 
        [[0.00035872, 0.00006676],
         [0.00006676, 0.00004139]],
 
        [[0.00017927, 0.00001578],
         [0.00001578, 0.0000106 ]],
 
        [[0.00078961, 0.00002952],
         [0.00002952, 0.00001533]]]),
 array([[[0.00017129, 0.00001643],
         [0.00001643, 0.00001212]],
 
        [[0.00036046, 0.00006673],
         [0.00006673, 0.00004149]],
 
        [[0.00017934, 0.00001579],
         [0.00001579, 0.0000106 ]],
 
        [[0.00079195, 0.00002988],
         [0.00002988, 0.0000155 ]]]),
 array([[[-0.00000084,  0.00000023],
         [ 0.00000023,  0.00000006]],
 
        [[ 0.00000174, -0.00000004],
         [-0.00000004,  0.0000001 ]],
 
        [[ 0.00000007,  0.        ],
         [ 0.        , -0.        ]],
 
        [[ 0.00000234,  0.00000036],
         [ 0.00000036,  0.00000016]]]))

In [7]:
keras.tree.map_stucture(np.cov, workflow.sample(conditions=samples, num_samples=1000))["parameters"]

NameError: name 'keras' is not defined

In [ ]:
estimates = workflow.estimate(conditions=samples)
cov_chol_inv = estimates["parameters"]["mvn"]["cov_chol_inv"]
np.linalg.inv(np.einsum("...ij,...ik->...jk", cov_chol_inv, cov_chol_inv))

In [ ]:
estimates